In [2]:
import pandas as pd
import duckdb
# Display all columns
pd.set_option('display.max_columns', None)

# Set max column width to a large value
pd.set_option('display.max_colwidth', None)

# Create a new DuckDB database file
con = duckdb.connect('balancer_dynamic_fee.duckdb')

In [3]:
block_gas_fees_mainnet = pd.read_csv('block_gas_fees_mainnet.csv')
block_gas_fees_mainnet.columns = ['block_number', 'gas_fee']
con.execute('CREATE TABLE block_fees_mainnet AS SELECT * FROM block_gas_fees_mainnet').fetchdf()

,Count
0,7033999


In [3]:
block_gas_fees_polygon = pd.read_csv('block_gas_fees_polygon.csv')

# Add column names
block_gas_fees_polygon = block_gas_fees_polygon.reset_index(drop=True)
block_gas_fees_polygon.index = block_gas_fees_polygon.index + 1
new_row = pd.DataFrame([['block_number', 'base_fee', 'priority_fee', 'gas_fee']], 
                       index=[0], 
                       columns=block_gas_fees_polygon.columns)
block_gas_fees_polygon = pd.concat([new_row, block_gas_fees_polygon]).reset_index(drop=True)
block_gas_fees_polygon.columns = block_gas_fees_polygon.iloc[0]
block_gas_fees_polygon = block_gas_fees_polygon.drop(block_gas_fees_polygon.index[0])
block_gas_fees_polygon = block_gas_fees_polygon.reset_index(drop=True)

block_gas_fees_polygon

,block_number,base_fee,priority_fee,gas_fee
0,40156252,96124467752,48473324648,144597792400
1,40156253,97824039546,50388051393,148212090939
2,40156254,101024752441,52248940332,153273692773
3,40156255,102258878848,47741121152,150000000000
4,40156256,103898320462,49723719576,153622040038
...,...,...,...,...
17498275,57659787,22,30999999978,31000000000
17498276,57659788,22,31999999978,32000000000
17498277,57659789,22,30000000000,30000000022
17498278,57659790,22,30000000000,30000000022


In [6]:
block_gas_fees_polygon = block_gas_fees_polygon[['block_number', 'gas_fee']]
con.execute('CREATE TABLE block_fees_polygon AS SELECT * FROM block_gas_fees_polygon').fetchdf()

,Count
0,17498280


In [7]:
# Commit the changes
con.commit()

# Close the connection
con.close()